In [ ]:
# Imported the necessary classes
import torch
import random
import torchvision
from torch import nn
from torchvision import transforms
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List, Tuple
from PIL import Image
import warnings
from typing import Optional, Tuple
from torch import Tensor
from torch.nn.init import constant_, xavier_normal_, xavier_uniform_
from torch.nn.parameter import Parameter
from torch.nn import functional as F
from torchvision.models import vision_transformer
from torch.nn.modules import activation
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# Get pretrained weights for ViT-Base
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT

# Setup a ViT model instance with pretrained weights
pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)

# Freeze the base parameters
for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False
    
# Function to find the percentage hits
# Takes two matricies, no of row zones, no of column zones, row population, column population and decimal place as input
def give_percentage(query, t_in_proj_weights, row_zones, col_zones, row_population, col_population, decimals):
    colx = query.shape[2]
    rowx = query.shape[1]
    B = query.shape[0]
    rowy = t_in_proj_weights.shape[0]
    coly = t_in_proj_weights.shape[1]
    no_pairs = row_zones*col_zones*colx*row_population*col_population # Total no of pairs

    pairs = torch.zeros(size=(no_pairs , 2))
    
    row_stride = rowx // row_zones
    col_stride = coly // col_zones
    row = []
    col = []
    
    # Randomly choosing pairs from each row zone and column zone
    for i in range(row_zones):
        row.extend(random.sample(range(i*row_stride , (i + 1)*row_stride), row_population))
    for i in range(col_zones):
        col.extend(random.sample(range(i*col_stride , (i + 1)*col_stride), col_population))
    curr = 0
    for i in range(B):
        for j in row:
            for k in range(colx):
                for l in col:
                    pairs[curr][0] = query[i][j][k]
                    pairs[curr][1] = t_in_proj_weights[k][l]
                    curr += 1
                    if(curr == no_pairs):
                        break
                if(curr == no_pairs):
                    break
            if(curr == no_pairs):
                break
        if(curr == no_pairs):
              break
    
    pairs = torch.abs(pairs) # Taking mod of weights as sign doesn’t matter
    pairs = pairs.cpu().numpy() # converting the matrix into numpy array for easy handling
    
    setp = set() # Set of pairs string
    unique = set()  # set of unique string
    
    # Converting pairs into string and then adding them into a set to remove the same pairs
    # Reversing the second number and concatenating it with the first one
    for i in range(pairs.shape[0]):
        first = f'{pairs[i][0]}'
        second = f'{pairs[i][1]}'
        final = first + second[::-1]
        setp.add(final)
    
    # Adding the reversed string again into the set after removing it once to remove the same but unordered pairs
    while len(setp) > 0:
        x = random.sample(setp , 1)
        curr = x[0]
        setp.remove(curr)
        rev = curr[::-1]
        setp.add(rev)
        setp.remove(rev)
        unique.add(str(curr))
    return (1 - len(unique)/no_pairs)*100 # Returns the percentage of pairs removed


# Main function to be called by user for finding percentage hits
# Takes model, encoders, decimals upto which round off is required, path of image as input
def find_perc_and_prob(
    model: torch.nn.Module,
    encoders : List[int],
    decimals : int,
    image_path: str,
    image_size: Tuple[int, int] = (224, 224),
    transform: torchvision.transforms = None,
    device: torch.device = device,
):
    # Opens image
    img = Image.open(image_path)

    # Create transformation for image (if one doesn't exist)
    if transform is not None:
        image_transform = transform
    else:
        image_transform = transforms.Compose(
            [
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    # Make sure the model is on the target device
    model.to(device)

    # Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
        
        # Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = image_transform(img).unsqueeze(dim=0)
        sample_image  = transformed_image.to(device)
        target_image_pred = model(transformed_image.to(device))
        final_out = model(sample_image)
        
        # Calling the functions manually as we want to temper the data in between
        out = model._process_input(sample_image)
        n = out.shape[0]
        batch_class_token = model.class_token.expand(n, -1, -1) # Expand the class token to the full batch
        out = torch.cat([batch_class_token, out], dim=1)
        input = out
        torch._assert(input.dim() == 3, f"Expected (batch_size, seq_length, hidden_dim) got {input.shape}")
        input = input + model.encoder.pos_embedding
        input = model.encoder.dropout(input)
        
        # For all the 12 encoders of ViT Base-16
        for i in range(12):
            percentage_avg = 0 # Setting percentage hits to 0
            torch._assert(input.dim() == 3, f"Expected (batch_size, seq_length, hidden_dim) got {input.shape}")
            encoder_block = model.encoder.layers[i]
            x = encoder_block.ln_1(input)
            
            # Multi head attention layer
            mha = encoder_block.self_attention
            if(i in encoders):
                x = torch.round(x , decimals = decimals)
                mha.in_proj_weight.data = torch.round(mha.in_proj_weight.data , decimals = decimals) # Rounding off weights to a particular decimal place
                
            # Finding the percentage hits in MHA layer for given values of row zones, column zones, row population and column population
            percentage_avg = give_percentage(x , mha.in_proj_weight.t() , 4 , 4 , 10 , 10 ,  decimals = decimals)
            x, _ = encoder_block.self_attention(x, x, x, need_weights=False)
            x = encoder_block.dropout(x)
            x = x + input
            y = encoder_block.ln_2(x)
            if(i in encoders):
                cnt = 0
                
                # For linear layers of MLP block
                for m in encoder_block.mlp.modules():
                    if isinstance(m, nn.Linear):
                        cnt = cnt + 1
                        m.weight.data = torch.round(m.weight.data , decimals = decimals) # Rounding off weights to a particular decimal place
                        y = torch.round(y , decimals = decimals)
                        
                        # Finding the percentage hits in linear layers of MLP block
                        if (cnt == 1):
                            percentage_avg  = percentage_avg + give_percentage( y , m.weight.t() , 4 , 4 , 10 , 10 ,  decimals = decimals)
                        else:
                            percentage_avg  = percentage_avg + give_percentage( y , m.weight , 4 , 4 , 10 , 10 ,  decimals = decimals)
                            
            # Printing the percentage hits of every encoder             
            print("percentage of ",i,"th encoder : " , percentage_avg/3 , "\n")   
            
            y = encoder_block.mlp(y)
            input = x + y 
   
        #Execute rest of layers as it is
        final_out = model.encoder.ln(input)
        final_out = final_out[:, 0]
        final_out = model.heads(final_out)
        print("Final output:: ")

    # Converts logits to prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)
    target_image_pred_probs_approx = torch.softmax(final_out, dim=1)

    # Convert prediction probabilities to prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)
    target_image_pred_label_approx = torch.argmax(target_image_pred_probs_approx, dim=1)
    
    # Plot image with predicted label and probability
    plt.figure()
    plt.imshow(img)
    plt.title(
        f"Pred: {target_image_pred_label} | Prob: {target_image_pred_probs.max():.3f}"
        f"Pred: {target_image_pred_label_approx} | Prob: {target_image_pred_probs_approx.max():.3f}"
    )
    plt.axis(False)

encoders = [0,1,2,3,4,5,6,7,8,9,10,11] # Encoders to be tempered

find_perc_and_prob(model=pretrained_vit, # Model
                    image_path = "./data.JPEG", # Path to the image
                    encoders = encoders, 
                    decimals = 2) # Set the decimal place to be rounded off